In [1]:
import jax
import jax.numpy as jnp
from jax import jit
from jax.config import config
config.update("jax_enable_x64", True)

In [2]:
sys.path.append('../jax_healpix/')
# from SPHT_jax import *
from YLM_jax_log import *
from YLM_jax import *
from utils import *
from reshape_utils import *

In [3]:
from scipy.special import binom, jn, loggamma
from scipy.special import eval_jacobi as jacobi

In [4]:
shift=1.e-2
theta=jnp.linspace(0+shift,jnp.pi/2-shift,20)
beta=jnp.cos(theta)
beta_s = jnp.sin(jnp.arccos(beta))

In [5]:
log_beta=jnp.log(beta)

In [6]:
log_beta_s, _ = logsumexp(
        2 * log_beta,
        jnp.zeros_like(log_beta),
        -1 * jnp.ones_like(log_beta),
        jnp.ones_like(log_beta),
    )  # sin^2=1-cos^2 = 1-beta^2
log_beta_s *= 0.5  # sqrt

In [7]:
log_beta_s<-100

DeviceArray([False, False, False, False, False, False, False, False,
             False, False, False, False, False, False, False, False,
             False, False, False, False], dtype=bool)

In [8]:
jnp.log(beta_s)

DeviceArray([-4.60518685e+00, -2.39149577e+00, -1.75807457e+00,
             -1.37788015e+00, -1.10814827e+00, -9.01331246e-01,
             -7.35669950e-01, -5.99384868e-01, -4.85388641e-01,
             -3.89096752e-01, -3.07385116e-01, -2.38041836e-01,
             -1.79456572e-01, -1.30434141e-01, -9.00775725e-02,
             -5.77122071e-02, -3.28352749e-02, -1.50820163e-02,
             -4.20307625e-03, -5.00008334e-05], dtype=float64)

In [9]:
l_max=500
l=np.arange(l_max)
# l=np.arange(900,l_max)
m1=0
m2=l*1.

In [10]:
spin_max=2

In [11]:
%time ylm=sYLM_recur(beta=beta,spins=(0,2),l_max=l_max-1)
%time ylm_log=sYLM_recur_log(log_beta=jnp.log(beta),spins=(0,2),l_max=l_max-1)

CPU times: user 4.12 s, sys: 1.91 s, total: 6.03 s
Wall time: 1.15 s
CPU times: user 4.88 s, sys: 304 ms, total: 5.19 s
Wall time: 1.28 s


In [12]:
jnp.log(beta)

DeviceArray([-5.00008334e-05, -4.20307625e-03, -1.50820163e-02,
             -3.28352749e-02, -5.77122071e-02, -9.00775725e-02,
             -1.30434141e-01, -1.79456572e-01, -2.38041836e-01,
             -3.07385116e-01, -3.89096752e-01, -4.85388641e-01,
             -5.99384868e-01, -7.35669950e-01, -9.01331246e-01,
             -1.10814827e+00, -1.37788015e+00, -1.75807457e+00,
             -2.39149577e+00, -4.60518685e+00], dtype=float64)

In [13]:
ylm2=ylm_log

In [14]:
lt=3
mt=1
ylm[0][lt,mt,:],ylm2[0][lt,mt,:]#,ylm_log[0][lt,mt,:]

(DeviceArray([-0.01292538, -0.11703697, -0.21455807, -0.30001706,
              -0.36865702, -0.41671721, -0.44165529, -0.44229673,
              -0.41890288, -0.37315332, -0.30804381, -0.22770578,
              -0.13715819, -0.04200664,  0.05189244,  0.13878239,
               0.21335072,  0.27104697,  0.30835484,  0.32300245],            dtype=float64),
 DeviceArray([-0.01292538, -0.11703697, -0.21455807, -0.30001706,
              -0.36865702, -0.41671721, -0.44165529, -0.44229673,
              -0.41890288, -0.37315332, -0.30804381, -0.22770578,
              -0.13715819, -0.04200664,  0.05189244,  0.13878239,
               0.21335072,  0.27104697,  0.30835484,  0.32300245],            dtype=float64))

In [15]:
log_Alm = log_A_lm(3, 1)
log_Alm_prev = log_A_lm(3 - 1, 1)

In [16]:
log_Alm,log_Alm_prev

(DeviceArray(0.73795326, dtype=float64),
 DeviceArray(0.80471896, dtype=float64))

In [17]:
xx=jnp.isclose(ylm[0],ylm2[0])
jnp.all(xx)

DeviceArray(True, dtype=bool)

In [18]:
jnp.where(~xx)

(DeviceArray([], dtype=int64),
 DeviceArray([], dtype=int64),
 DeviceArray([], dtype=int64))

In [19]:
jnp.all(ylm[0][jnp.where(~xx)]==0)

DeviceArray(True, dtype=bool)

In [20]:
ylm2[0][jnp.where(~xx)]

DeviceArray([], dtype=float64)

In [21]:
jnp.where(~xx)

(DeviceArray([], dtype=int64),
 DeviceArray([], dtype=int64),
 DeviceArray([], dtype=int64))

In [22]:
jnp.unique(jnp.where(~xx)[2])

DeviceArray([], dtype=int64)

In [23]:
s=2
xx=jnp.isclose(ylm[s][:,:,0],ylm2[s][:,:,0])
jnp.all(xx)

DeviceArray(True, dtype=bool)

In [24]:
s=-2
xx=jnp.isclose(ylm[s][:,:,0],ylm2[s][:,:,0])
jnp.all(xx)

DeviceArray(True, dtype=bool)

In [25]:
lt,mt=jnp.where(~xx)

In [26]:
ylm2[s][jnp.where(~xx)]

DeviceArray([], dtype=float64)

In [27]:
ylm[s][jnp.where(~xx)]

DeviceArray([], dtype=float64)

In [28]:
s=2
lt=2
mt=2
th_i=10

alm=alpha_lm(lt,mt)
alm_log=log_alpha_lm(lt,mt)

ylm[s][lt,mt,th_i],ylm[s][lt,mt,th_i]/(np.sqrt(2*lt+1)/np.sqrt(4*np.pi))

(DeviceArray(0.23011518, dtype=float64),
 DeviceArray(0.36480872, dtype=float64))

In [29]:
beta[th_i],theta[th_i]

(DeviceArray(0.6776687, dtype=float64), DeviceArray(0.82620859, dtype=float64))

In [30]:
def log_binom(x,y):
    return loggamma(x+1)-loggamma(y+1)-loggamma(x-y+1)
def wigner_d2(m1, m2, theta, l, l_use_bessel=1.0e3):
    """
    Function to compute wigner small-d matrices.
    This maybe less accurate than our main jax functions which use recursion.
    """
    l=l[:,None,None]
    m2=m2[None,:,None]
    m1=m1*np.ones_like(m2)
    theta=theta[None,None,:]
    k = np.amin([l - m1, l - m2, l + m1, l + m2], axis=0)
    a = np.absolute(m1 - m2)
    lamb = m2 - m1  # lambda
    x=m2<=m1
    lamb[x]=0
    b = 2 * l - 2 * k - a

#     d_mat = np.sqrt(
#         binom(2 * l - k, k + a)
#     )
    d_mat=0.5*log_binom(2 * l - k, k + a)
    
#     d_mat /= np.sqrt(binom(k + b, b))
    d_mat-=0.5*log_binom(k + b, b)
    x = k+b < b
    d_mat[x] = -np.inf
  
    beta_s=np.sin(theta / 2.0)
    beta=np.cos(theta / 2.0)
    
    d_mat=d_mat+a*np.log(np.absolute(beta_s))+b*np.log(np.absolute(beta))
    sign_dmat=np.sign(beta_s)**a
    sign_dmat=sign_dmat*(np.sign(beta)**b)
    
    d_mat=sign_dmat*np.exp(d_mat)
#     d_mat[x]=0
    x = d_mat == 0
    d_mat *= jacobi(k, a, b, np.cos(theta))  # l
    
    d_mat*=(-1)**lamb[0,:,0][None,:,None]
    
    d_mat[x] = 0

    if l_use_bessel is not None:
        x = l[:,0,0] >= l_use_bessel

        d_mat[x,:,:]=jn(m1-m2,l[x,:,:]*theta)
    return d_mat


In [31]:
s=2
lt=2
mt=2
th_i=10

alm=alpha_lm(lt,mt)

wd=wigner_d2(s, np.atleast_1d(mt), np.atleast_1d(theta[th_i]), np.atleast_1d(lt))
wd2=wigner_d2(-s, np.atleast_1d(mt), np.atleast_1d(theta[th_i]), np.atleast_1d(lt))

ylm_t=wd*np.sqrt(2*lt+1)/np.sqrt(4*np.pi)
ylm_t2=wd2*np.sqrt(2*lt+1)/np.sqrt(4*np.pi)

wd,wd2,wd+wd2,beta[th_i],theta[th_i]
#https://www.wolframalpha.com/input?i=WignerD%5B%7B2%2C2%2C-2%7D%2C0%2C0.82620859%2C0%5D%2BWignerD%5B%7B2%2C2%2C2%7D%2C0%2C0.82620859%2C0%5D

(array([[[0.70364307]]]),
 array([[[0.02597437]]]),
 array([[[0.72961743]]]),
 DeviceArray(0.6776687, dtype=float64),
 DeviceArray(0.82620859, dtype=float64))

In [32]:
ylm[s][lt,mt,th_i]+ylm[-s][lt,mt,th_i], ylm[s][lt,mt,th_i]-ylm[-s][lt,mt,th_i]

(DeviceArray(0.01638419, dtype=float64),
 DeviceArray(0.44384618, dtype=float64))

In [33]:
ylm_t,ylm_t2

(array([[[0.44384618]]]), array([[[0.01638419]]]))

In [35]:
ylm[s][lt,mt,th_i]/((ylm_t+ylm_t2)/2),((ylm_t+ylm_t2)/2)

(DeviceArray([[[1.]]], dtype=float64), array([[[0.23011518]]]))

In [36]:
ylm[0][lt,mt,th_i]*(2*(mt**2-lt)-(lt*(lt-1)))/beta_s[th_i]**2 #+ ylm[0][lt-1,mt,th_i]*(2*alm*beta[th_i]/beta_s[th_i]**2)

DeviceArray(0.7725484, dtype=float64)

In [37]:
ylm[s][lt,mt,th_i]

DeviceArray(0.23011518, dtype=float64)

In [38]:
ylm[-s][lt,mt,th_i]

DeviceArray(-0.21373099, dtype=float64)

In [39]:
from jax.scipy.special import gammaln as loggamma

In [40]:
loggamma(10000)

DeviceArray(82099.71749644, dtype=float64)